In [14]:
import numpy as np
import json
import os
import sys 
import csv

In [15]:
label_set = np.loadtxt('./data/custom_labels.csv', delimiter=',', dtype='str')

In [16]:
datafiles_dir = './data/datafiles'

In [17]:
if os.path.exists(datafiles_dir) == False:
    os.mkdir(datafiles_dir)

In [18]:
meta = np.loadtxt('./data/custom_data/custom_meta_ncd.csv', delimiter=',', dtype=str, skiprows = 1)

In [19]:
meta

array([['693048.wav', '4', '0', 'lesred1', '20.0', '1 1 1 1'],
       ['692820.wav', '4', '0', 'lesred1', '23.312875', '1 1 1 1 1'],
       ['692819.wav', '5', '0', 'lesred1', '17.504875', '1 1 1 1'],
       ...,
       ['559833.wav', '5', '19', 'sposan', '19.45225', '1 1 1 1'],
       ['254627.wav', '4', '19', 'sposan', '1.700625', '0'],
       ['211173.wav', '5', '19', 'sposan', '3.8355', '1']], dtype='<U2565')

In [20]:
nocall_label = 'a020'
for fold in [1,2,3,4,5]:
    base_path = "/datasets/xeno_canto/sm_dataset/"
    #meta = np.loadtxt('./data/custom_data/custom_meta_duration_probs.csv', delimiter=',', dtype='str', skiprows=1)
    
    train_wav_list = []
    eval_wav_list = []
    
    for i in range(0, len(meta)):   # range(0, len(meta)) 200 was small dataset
        cur_labels = [x for x in meta[i][2].split("-")]
        cur_labels = ['a'+x.zfill(3) for x in cur_labels]
        if len(cur_labels) > 0:
            sec_labels = cur_labels[1:]
        else:
            sec_labels = []
        # indices shifted by 1
        #cur_path = (meta[i][1]).split('.')[0] + '.pt'
        cur_path = meta[i][0]
        
        cur_fold = int(meta[i][1])
        curr_call_detection = (meta[i][5]).split()
        for cd_ind, cd_value in enumerate (curr_call_detection):

            if cd_value == '1':
                cur_dict = {"wav": base_path + cur_path, "labels": "-".join(cur_labels), "sec_labels":sec_labels,"segment":cd_ind,}
                
                if cur_fold == fold:
                    eval_wav_list.append(cur_dict)
                else:
                    train_wav_list.append(cur_dict)
            elif cd_value == '0':
                sec_labels = []
                cur_dict = {"wav": base_path + cur_path, "labels": nocall_label, "sec_labels":sec_labels ,"segment":cd_ind} #
                if cur_fold == fold:
                    eval_wav_list.append(cur_dict)
                else:
                    train_wav_list.append(cur_dict)
        
    print('fold {:d}: {:d} training samples, {:d} test samples'.format(fold, len(train_wav_list), len(eval_wav_list)))

    with open(datafiles_dir + '/custom_train_data_'+ str(fold) +'.json', 'w') as f:
        json.dump({'data': train_wav_list}, f, indent=1)

    with open(datafiles_dir + '/custom_eval_data_'+ str(fold) +'.json', 'w') as f:
        json.dump({'data': eval_wav_list}, f, indent=1)   
print('Finished custom Preparation')

fold 1: 146364 training samples, 39276 test samples
fold 2: 151553 training samples, 34087 test samples
fold 3: 147722 training samples, 37918 test samples
fold 4: 146721 training samples, 38919 test samples
fold 5: 150200 training samples, 35440 test samples
Finished custom Preparation


In [4]:
with open('./data/datafiles/custom_train_data_1.json', 'r') as fp:
            data_json = json.load(fp)

In [7]:
datum = data_json['data'][0]


In [8]:
datum

{'wav': '/datasets/xeno_canto/sm_dataset/693048.wav',
 'labels': 'a000',
 'sec_labels': [],
 'segment': 0}

In [10]:
sec_labels = datum['sec_labels']

In [11]:
for sec_label in sec_labels:
            label_weights[int(self.index_dict[sec_label])] = 0.0